In [1]:
import numpy as np
import nrrd
import nibabel as nib

In [2]:
def correct_header(header):
    affine = np.zeros((3, 3), dtype="float")
    affine[0:3, 0:3] = -header["space directions"]
    affine[2, 2] = -affine[2, 2]

    header["space directions"] = np.copy(affine)

    affine = -header["space origin"]
    affine[2] = -affine[2]

    header["space origin"] = np.copy(affine)


In [3]:
from skimage.filters import frangi
import nrrd
import numpy as np


def save_frangi(filename: str, target_filename: str, sigmas=[0.5, 0.7, 0.9, 1.1, 1.3]):
    data, header = nrrd.read(filename)
    inverted_data = np.max(data) - data - 1024

    filtered_image = frangi(inverted_data, sigmas=sigmas)
    filtered_image = filtered_image * 1000 / np.max(filtered_image)

    nrrd.write(target_filename, filtered_image, header)


In [2]:
import nibabel as nib
import nrrd
import numpy as np


def nrrd_to_nifti(filename, target_filename):
    data, header = nrrd.read(filename)

    affine = np.zeros((4, 4), dtype="float")
    affine[0:3, 0:3] = -header["space directions"]
    affine[3, 3] = 1
    affine[2, 2] = -affine[2, 2]

    affine[0:3, 3] = -header["space origin"]
    affine[2, 3] = -affine[2, 3]

    img = nib.Nifti1Image(data, affine)

    nib.save(img, target_filename)

    return affine


In [1]:
import os
from scancovia import AiSegment


def save_lungs_seg(filename: str, target_filename: str):

    _, header = nrrd.read(filename)

    nrrd_to_nifti(filename, os.path.splitext(filename)[0] + ".nii.gz")
    filename = filename.replace(".nrrd", ".nii.gz")

    ai_segment = AiSegment()
    output = ai_segment(filename)

    nrrd.write(target_filename, output["lungs_mask"].transpose(1, 0, 2), header)

    os.remove(filename)

    return output


ModuleNotFoundError: No module named 'scancovia'

In [8]:
import nrrd

def nrrd_get_data(filename):
    data, _ = nrrd.read(filename)
    return data

In [9]:
import numpy as np

def get_emboly_coverage(emboly_seg_path: str, vessels_seg_path: str, threshold=None):

    ground_truth = np.where(nrrd_get_data(emboly_seg_path) > 0, 1, 0)

    vessels_seg = nrrd_get_data(vessels_seg_path)

    if threshold is not None:
        vessels_seg = np.where(vessels_seg > threshold, 1, 0)

    return np.sum(ground_truth * vessels_seg) / np.sum(ground_truth)


In [14]:
import numpy as np

def get_recall(true_seg_path: str, vessels_seg_path: str, threshold=None):

    ground_truth = np.where(nrrd_get_data(true_seg_path) > 0, 1, 0)

    vessels_seg = nrrd_get_data(vessels_seg_path)

    if threshold is not None:
        vessels_seg = np.where(vessels_seg > threshold, 1, 0)
    else:
        vessels_seg = np.where(vessels_seg > 0, 1, 0)

    return np.sum(ground_truth * vessels_seg) / np.sum(ground_truth)


In [19]:
threshold = 1

print(get_recall("../data/CAD_PE/rs/0005RefStd.nrrd", "./data/005_frangi.nrrd", threshold=threshold))
print(get_recall("./data/005_lungs.nrrd", "./data/005_frangi.nrrd", threshold=threshold))

0.22693959640032724
0.08963314980475266


In [17]:
get_recall("./data/005_lungs.nrrd", "./data/005_frangi.nrrd", threshold=0.01)

0.21793437974779326

In [16]:
save_frangi("../data/CAD_PE/images/005.nrrd", "./005_frangi.nrrd")

In [17]:
output = save_lungs_seg("../data/CAD_PE/images/005.nrrd", "./005_lungs.seg.nrrd")

In [18]:
data, header = nrrd.read("../data/CAD_PE/images/010.nrrd")
data_emb, header_emb = nrrd.read("../data/CAD_PE/rs/0010RefStd.nrrd")

In [27]:
import vtk

def readnrrd(filename):
    """Read image in nrrd format."""
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(filename)
    reader.Update()
    info = reader.GetInformation()
    return reader.GetOutput(), info

def writenifti(image,filename, info):
    """Write nifti file."""
    writer = vtk.vtkNIFTIImageWriter()
    writer.SetInputData(image)
    writer.SetFileName(filename)
    writer.SetInformation(info)
    writer.Write()

In [31]:
m, info = readnrrd("../data/CAD_PE/images/010.nrrd")
writenifti(m, './data/010.nii.gz', info)

In [5]:
nrrd_to_nifti("../data/CAD_PE/rs/0010RefStd.nrrd", "./data/0010RefStd_custom.nii.gz")

array([[  -0.65625 ,   -0.      ,   -0.      ,   62.015625],
       [  -0.      ,   -0.65625 ,   -0.      ,  151.421875],
       [  -0.      ,   -0.      ,    0.5     , -101.5     ],
       [   0.      ,    0.      ,    0.      ,    1.      ]])

In [30]:
m, info = readnrrd("../data/CAD_PE/rs/0010RefStd.nrrd")
writenifti(m, './data/0010RefStd.nii.gz', info)

2021-12-16 01:42:39.000 ( 144.762s) [           747A8]      vtkNrrdReader.cxx:575   WARN| vtkNrrdReader (0x110d61160): Unknown field: 'Segmentation_ConversionParameters:=Collapse labelmaps|1|Merge the labelmaps into as few shared labelmaps as possible 1 = created labelmaps will be shared if possible without overwriting each other.&Compute surface normals|1|Compute surface normals. 1 (default) = surface normals are computed. 0 = surface normals are not computed (slightly faster but produces less smooth surface display).&Crop to reference image geometry|0|Crop the model to the extent of reference geometry. 0 (default) = created labelmap will contain the entire model. 1 = created labelmap extent will be within reference image extent.&Decimation factor|0.0|Desired reduction in the total number of polygons. Range'


In [ ]:
m, info = readnrrd("../data/CAD_PE/lungs/010_lungs.seg.nrrd")
writenifti(m, './data/010_lungs.seg.nii.gz', info)

In [22]:
import vtk 

reader = vtk.vtkNrrdReader()
reader.SetFileName("../data/CAD_PE/rs/0010RefStd.nrrd")
reader.Update()
info = reader.GetInformation()
reader.GetOutput(), info

2021-12-16 01:40:16.143 (   1.900s) [           747A8]      vtkNrrdReader.cxx:575   WARN| vtkNrrdReader (0x1237d3200): Unknown field: 'Segmentation_ConversionParameters:=Collapse labelmaps|1|Merge the labelmaps into as few shared labelmaps as possible 1 = created labelmaps will be shared if possible without overwriting each other.&Compute surface normals|1|Compute surface normals. 1 (default) = surface normals are computed. 0 = surface normals are not computed (slightly faster but produces less smooth surface display).&Crop to reference image geometry|0|Crop the model to the extent of reference geometry. 0 (default) = created labelmap will contain the entire model. 1 = created labelmap extent will be within reference image extent.&Decimation factor|0.0|Desired reduction in the total number of polygons. Range'


(<vtkmodules.vtkCommonDataModel.vtkImageData(0x123773c00) at 0x1605598e0>,
 <vtkmodules.vtkCommonCore.vtkInformation(0x60000273c1c0) at 0x160559880>)

In [26]:
info

<vtkmodules.vtkCommonCore.vtkInformation(0x60000273c1c0) at 0x160559880>

In [19]:
print(data.shape)
print(data_emb.shape)

(512, 512, 426)
(186, 45, 20)


In [20]:
header_emb

OrderedDict([('type', 'unsigned int'),
             ('dimension', 3),
             ('space', 'left-posterior-superior'),
             ('sizes', array([186,  45,  20])),
             ('space directions',
              array([[0.65625, 0.     , 0.     ],
                     [0.     , 0.65625, 0.     ],
                     [0.     , 0.     , 0.5    ]])),
             ('kinds', ['domain', 'domain', 'domain']),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('space origin', array([ -62.015625, -151.421875, -101.5     ])),
             ('Segment0_Color', '0.901961 0.862745 0.27451'),
             ('Segment0_ColorAutoGenerated', '1'),
             ('Segment0_Extent', '0 185 0 44 0 19'),
             ('Segment0_ID', 'Segment_1'),
             ('Segment0_LabelValue', '1'),
             ('Segment0_Layer', '0'),
             ('Segment0_Name', 'fat'),
             ('Segment0_NameAutoGenerated', '1'),
             ('Segment0_Tags',
              'TerminologyEnt

In [21]:
header

OrderedDict([('type', 'short'),
             ('dimension', 3),
             ('space', 'left-posterior-superior'),
             ('sizes', array([512, 512, 426])),
             ('space directions',
              array([[0.65625, 0.     , 0.     ],
                     [0.     , 0.65625, 0.     ],
                     [0.     , 0.     , 0.5    ]])),
             ('kinds', ['domain', 'domain', 'domain']),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('space origin', array([-167.671875, -324.671875, -239.      ]))])

In [60]:
nrrd.write("010_lung_mask.seg.nrrd", output["lungs_mask"].transpose(1,0,2), header)

In [39]:
img = nib.Nifti1Image(output["lungs_mask"], affine)

nib.save(img, "010_lungs_mask.seg.nii")

In [15]:
from scancovia import AiSegment

ai_segment = AiSegment() 
output = ai_segment('010.nii')

In [20]:
data.shape

(512, 512, 426)

In [30]:
np.max(output["lungs_mask"])

2

In [37]:
affine = nrrd_to_nifti("../data/CAD_PE/images/010.nrrd", "./010.nii")

In [12]:
from src.frangi import find_best_threshold
import os

path_root = os.path.dirname(os.getcwd())

find_best_threshold(
    [os.path.join(path_root, "data/CAD_PE/frangi/010_frangi.nrrd")],
    [os.path.join(path_root, "data/CAD_PE/lungs_scancovia/010_lungs.seg.nrrd")],
    [os.path.join(path_root, "data/CAD_PE/rs/0010RefStd.nrrd")],
)


/Users/corentinberteaux/Documents/3A_CS/Projet CVN/cvn-pulmonary-embolism


ValueError: operands could not be broadcast together with shapes (186,45,20) (512,512,426) 

In [19]:
from src.utils import nrrd_get_data
import nrrd

data, header = nrrd.read(os.path.join(path_root, "data/CAD_PE/frangi/010_frangi.nrrd"))
print(header)
print(data.shape)
print("----------------")

data, header = nrrd.read(os.path.join(path_root, "data/CAD_PE/lungs_scancovia/010_lungs.seg.nrrd"))
print(header)
print(data.shape)
print("----------------")

data, header = nrrd.read(os.path.join(path_root, "data/CAD_PE/rs/0010RefStd.nrrd"))
print(header)
print(data.shape)

OrderedDict([('type', 'double'), ('dimension', 3), ('space', 'left-posterior-superior'), ('sizes', array([512, 512, 426])), ('space directions', array([[0.65625, 0.     , 0.     ],
       [0.     , 0.65625, 0.     ],
       [0.     , 0.     , 0.5    ]])), ('kinds', ['domain', 'domain', 'domain']), ('endian', 'little'), ('encoding', 'gzip'), ('space origin', array([-167.671875, -324.671875, -239.      ]))])
(512, 512, 426)
----------------
OrderedDict([('type', 'int64'), ('dimension', 3), ('space', 'left-posterior-superior'), ('sizes', array([512, 512, 426])), ('space directions', array([[0.65625, 0.     , 0.     ],
       [0.     , 0.65625, 0.     ],
       [0.     , 0.     , 0.5    ]])), ('kinds', ['domain', 'domain', 'domain']), ('endian', 'little'), ('encoding', 'gzip'), ('space origin', array([-167.671875, -324.671875, -239.      ]))])
(512, 512, 426)
----------------
OrderedDict([('type', 'unsigned int'), ('dimension', 3), ('space', 'left-posterior-superior'), ('sizes', array([186

ValueError: cannot reshape array of size 167400 into shape (512,512,426)